In [1]:
import pandas as pd
import numpy as np
import itertools
import urllib2
import json
from tqdm import tqdm
pd.set_option('max_columns', None)

In [2]:
week = 14

In [3]:
team_map = {1: 'Arsenal',
    2: 'Bournemouth',
    3: 'Burnley',
    4: 'Chelsea',
    5: 'Crystal Palace',
    6: 'Everton',
    7: 'Hull City',
    8: 'Leicester City',
    9: 'Liverpool',
    10: 'Manchester City',
    11: 'Manchester United',
    12: 'Middlesbrough',
    13: 'Southampton',
    14: 'Stoke City',
    15: 'Sunderland',
    16: 'Swansea City',
    17: 'Tottenham Hotspur',
    18: 'Watford',
    19: 'West Bromwich Albion',
    20: 'West Ham United'}

In [4]:
def gen_fixtures(week):
    '''Function for generating fixtures for given week'''
    
    fixurl = 'https://fantasy.premierleague.com/drf/fixtures/?event={}'
    fixlist=[]
    resp = urllib2.urlopen(fixurl.format(week))
    fixtures = json.loads(resp.read())
    resp.close()

    for fixture in fixtures:
        fixlist.append((fixture['team_h'], fixture['team_a']))

    return (pd.DataFrame(fixlist, columns=['hometeam','awayteam'])
     .replace(team_map))

In [5]:
def gen_ptstable(week):
    '''Function to generate points table as it stands before start of week'''
    
    taburl = 'https://footballapi.pulselive.com/football/standings?compSeasons=54&altIds=true&detail=2&FOOTBALL_COMPETITION=1&gameweekNumbers=1-{}'
        
    request = urllib2.Request(taburl.format(week-1), headers= {'Origin': 'https://www.premierleague.com'})
    contents = urllib2.urlopen(request).read()
    entry = json.loads(contents)['tables'][0]['entries']
    
    cur = []
    for row in entry:
        cur.append((row['team']['name'], int(row['overall']['points']), int(row['position'])))
    
    return pd.DataFrame(cur, columns=['team','curpts','curpos']).set_index('team')

In [6]:
def gen_posrange(curpos, nextfix):
    '''Function to generate range of positions for a team
    given their current position and next round of fixtures'''
    
    futpts = {}
    ctr = 1
    for i in tqdm(itertools.product(['H','A','D'], repeat= len(nextfix))):
        for j in range(len(nextfix)):
            hp = np.where(i[j] == 'H', 3, np.where(i[j] == 'A', 0, 1))
            ap = np.where(i[j] == 'A', 3, np.where(i[j] == 'H', 0, 1))
            futpts[nextfix.loc[j, 'hometeam']] = curpos.loc[nextfix.loc[j, 'hometeam'], 'curpts'] + hp
            futpts[nextfix.loc[j, 'awayteam']] = curpos.loc[nextfix.loc[j, 'awayteam'], 'curpts'] + ap
        curpos = curpos.join(pd.Series(futpts, name='{}'.format(ctr)))
        ctr += 1

    # curbkup = cur.copy()
    # cur.head()

    # curbkup.shape

    posrange = (curpos.drop(['curpts', 'curpos'], axis=1)
     .rank(method='min', ascending=False).
     join(curpos.drop(['curpts', 'curpos'], axis=1)
          .rank(method='max', ascending=False), lsuffix='min', rsuffix='max'))

    return (pd.DataFrame([posrange.min(axis=1), posrange.max(axis=1)],
                            index=['minpos','maxpos']).transpose().
                join(curpos.curpos))

In [7]:
nextfix = gen_fixtures(week)
curpos = gen_ptstable(week)
posfinal = gen_posrange(curpos, nextfix)

59049it [28:58, 22.71it/s]


In [8]:
posfinal.to_csv('data/posfinal_{}.csv'.format(week))
print posfinal

                      minpos  maxpos  curpos
team                                        
Chelsea                  1.0     4.0       1
Liverpool                1.0     4.0       2
Manchester City          1.0     4.0       3
Arsenal                  2.0     4.0       4
Tottenham Hotspur        5.0     5.0       5
Manchester United        6.0     9.0       6
Everton                  6.0    10.0       7
Watford                  6.0    12.0       8
West Bromwich Albion     7.0    12.0       9
Southampton              7.0    12.0      10
Stoke City               7.0    14.0      11
Bournemouth              9.0    16.0      12
Burnley                  9.0    17.0      13
Leicester City          12.0    17.0      14
Middlesbrough           12.0    19.0      15
West Ham United         12.0    19.0      16
Crystal Palace          13.0    20.0      17
Hull City               13.0    20.0      18
Swansea City            16.0    20.0      19
Sunderland              17.0    20.0      20
